 This is a free tutorial that you can use to explore the secondary structure of  the DNA sequence of
 any genome or partial-DNA sequence of your choice.At the time it will only work for UNIX machines and maybe OSX because the ViennaRNA package only works for those, if you have a Windows machine please proceed to use the Linux virtual machine. I hope this helps you out , if something gets broken please send me a message .


First you will need to install some packages and dependencys for data visualisation and handling.
For this example we will be using the complete genome sequence of the Human Herpes 1 type 17 virus with the ascencion number:NC_001806.2. The herpes 1 virus was chosen because it's genome consists of a  double-stranded linear DNA molecule. Other authors have reported that the herpes simplex viral genome does posses secondary DNA structures like G-quadruplexes. (Artusi,2016)

Secondary DNA structures found in double-stranded DNA genomes can be correlated with deletions.Joana Damas has reported the relationship of Non-B DNA structures with the amount of deletions found in the mithocondrial DNA genome (It is also a double-stranded DNA molecule). These deletions are important because 
when a mitDNA gene becomes deleted, then the cell may not be able to synthesize  efficiently a protein complex that is essential for the oxidative phosphorilation . In the case of viral genomes, these secondary DNA structures could be related to translocation events, and may induce genomic instabilities within fragile sites found at the human genome.


< >

Now that all has been said, Lets have some fun!

< >

Disclaimer: I don't claim rights nor I am the creator of ViennaRNA or any of their packages and
dependencies. The purpose of this python notebook is to facilitate the use of ViennaRNA for DNA secondary
structure prediction at the level of the whole genome of any Biological system. 
Some lines of code were based in the available tutorials that you can Download from the ViennaRNArepository for the prediction of RNA secondary structures.

Artusi, Sara, et al. "Visualization of DNA G-quadruplexes in herpes simplex virus 1-infected cells." Nucleic acids research 44.21 (2016): 10343-10353.

Gruber AR, Lorenz R, Bernhart SH, Neuböck R, Hofacker IL.
The Vienna RNA Websuite. Nucleic Acids Res. 2008 

## Please install this dependencies:


In [ ]:

#conda install seaborn
#conda install pandas
#conda install -c bioconda viennarna

We will now call the dependencies and packages we will be using.

## Calculating the Folding Energy Potential of a Sequence

In [23]:
import RNA
from RNA import params_load
# params_load is a function that comes with the ViennaRNA package. ViennaRNA was originaly created 
#for the exploration of RNA secondary structures and has RNA parameters as default, this is why 
#we need to load the DNA parameters of (Mathews,2004) to predict DNA secondary structures.
params_load('dna_mathews2004.par') 
RNA.cvar.dangles = 0
#If you want to consider dangling energies at both sides of any helix found in your sequence you 
#can set that parameter to '1'.

First let's call the sequence or the genome we want to analyze. The function readgenome opens up a.txt
File that has the FASTA file format. So, to use this function you need to download your target genome
and save it in a .txt file using the FASTA format.

In [ ]:
def readgenome(filename):
    genome =''
    with open(filename,'r')as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readgenome('Herpes1.txt')
len(genome)

 Afterwards, we will use the first 100 nucleotides of the herpes 1 simplex viral genome to predict the Secondary DNA structure most likely to form.The results should be identical to those generated when
you compute the minimum free energy(MFE) and dot bracket notation using the RNAfold Webserver with the same parameters we are using.

In [25]:
seq= genome[:100]
# creates a fold_compound data structure (required for all subsequently applied  algorithms)
fc = RNA.fold_compound(seq)

# compute MFE and MFE structure
(mfe_struct, mfe) = fc.mfe()

# rescale Boltzmann factors for partition function computation
fc.exp_params_rescale(mfe)


print("%s\n%s (%6.2f)" % (seq, mfe_struct, mfe))


AGCCCGGGCCCCCCGCGGGCGCGCGCGCGCGCAAAAAAGGCGGGCGGCGGTCCGGGCGGCGTGCGCGCGCGCGGCGGGCGTGGGGGGCGGGGCCGCGGGA
.((((.(.((((((((.(.(((((((((((((......(.((..(((....)))..)).)))))))))))))).)..))).))))).).))))....... (-44.20)


## Calculating the Folding Energy Potential of a Genome using a Sliding Window Analysis

Now that we know how to calculate the minimum free energy of a DNA sequence, we can try analyzing
the complete viral genome of the Herpes 1 simplex type 17. To achieve this, we will use a type of analysis
that is called "Sliding Window Analysis". We use the Sliding Window Analysis to explore how the folding energy potential
 changes through the entire genome. The folding energy potential of a sequence suggests that DNA can also fold
into compact three-dimensional structure. Note that Secondary DNA structures will tend to form within regions of the genome
that harbor the most negative folding energy potential.
In this case, we will use a sliding window of 500 nucleotides with a step size of 75.

In [29]:
fin=0
def sliding_window_analysis(genome,function, window_size=500, step_size=75):
    """Return an iterator that yields (start, end, property) tuples.
    Where start and end are the indices used to slice the input list
    and property is the return value of the function given the sliced
    list.
    """
    for start in range(0, len(genome), step_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end, function(genome[start:end])

    for start in range(0, len(genome), window_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end, function(genome[start:end])
        
        
def fold_energy(genome):
    'Return the  folding energy of the compund sliced sequence'
    import RNA
    from RNA import params_load
    params_load('dna_mathews2004.par') 
    RNA.cvar.dangles = 0
    
    fc = RNA.fold_compound(genome)
    (mfe_struct, mfe) = fc.mfe()
    fc.exp_params_rescale(mfe)
    (pp, pf) = fc.pf()
    comp_fe=pf
    fin=pf
    return fin
    

In [ ]:
for start, end, fin in sliding_window_analysis(genome,fold_energy):
    print(start,end,fin)

Then, we will save our results for further analysis.

In [ ]:
def save_csv(file):
    file ='local_foldE_500.csv'
    with open(file,'w')as file_handle:
        header = "start,middle,end,folding_energy_potential\n"
        file_handle.write(header)
        for start, end, fin in sliding_window_analysis(genome,fold_energy):
            middle = (start + end ) / 2
            row = "{},{},{},{}\n".format(start, middle, end, fin)
            file_handle.write(row) 
    return(file)   
files=''
save_csv(files)

## Calculating the number of deletions found at each Window 

 Now comes the fun part!.
 

To continue with the second part of our example, we will use again a sliding window analysis to count how many deletions are found within each previously calculated window. Later,we will save those results within a .CSV file and at the end we will plot the number of deletions found at each window against the folding energy potential of the same window.
 
 The number of deletions is calculated from  a Dummy list with 1000 random deletions that have a random frequency pattern. In real life, you should use your own list with your own annotatiions, and remember to take into consideration that any deletion has a starting position(5') within your chosen genome and an ending position(3'). This is why if you want to try to analyze your genome you should create two  separate lists containing either the starting position or the ending position. Consequently, you will end up with two .CSV files that will have each sliding window analysis to count the number of deletions at each window. Each list will have a different distribution pattern, because even if the amount of deletions won't change, the location of the (5') or the (3') position of the deletion wil be different.
 

Damas, Joana, et al. "Mitochondrial DNA deletions are associated with non-B DNA conformations." Nucleic acids research 40.16 (2012): 7606-7621.


In [ ]:

import pandas
df1 = pandas.read_csv('Dummy_deletions.csv')

Deletion_list = df1.Deletion.tolist()
Number_of_Deletions_list = df1.Number_of_Deletions.tolist()

zipped_list = list(zip(Deletion_list,Number_of_Deletions_list))
def sliding_window_analysis2(genome, window_size=500, step_size=75):
    """Return an iterator that yields (start, end, property) tuples.
    Where start and end are the indices used to slice the input list
    and property is the return value of the function given the sliced
    list.
    """
    for start in range(0, len(genome), step_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end

    for start in range(0, len(genome), window_size):
        end = start + window_size
        if end > len(genome):
            break
        yield start, end
             
#Creates an empty list
windows_list = []
for start,end in sliding_window_analysis(genome):
    seq = genome[start:end]
    windows_list.append((start,end))

#Creates an empty list
reads_sum = []
for window in windows_list:
    window_low = window[0]
    window_high = window[1]
#Sums the number of the deletions found at each window
    
    this_sum = 0
    for item in zipped_list:
        Deletions = item[0]
        read = item[1]
        if (Deletions >= window_low and Deletions <= window_high):
            this_sum += read
            
    reads_sum.append((window_low, window_high, this_sum))

for item in reads_sum:
    a=item[0]
    b=item[1]
    c=item[2]
    print(a,b,c)


In [ ]:
def save_csv(file):
    file ='Dummy_deletions_window.csv'
    with open(file,'w')as file_handle:
        header = "start,middle,end,Deletions\n"
        file_handle.write(header)
        for a, b, c in reads_sum:
            middle = (a + b ) / 2
            row = "{},{},{},{}\n".format(a, middle, b, c)
            file_handle.write(row) 
    return(file) 
files=''
save_csv(files)

## Plotting the results of both analysis

In [ ]:
import pandas as pd

df=pd.read_csv('Dummy_deletions_window.csv')
df3=pd.read_csv('local_foldE_500.csv')

middle_list = df.middle.tolist()
Deletions_list = df.Deletions.tolist()
foldE_list= df3.folding_energy_potential.tolist()
x=middle_list
y=Deletions_list
y1=foldE_list

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(x, y)
ax2.plot(x, y1, '-', color="red" )

ax1.set_xlabel('Number of Deletions vs the Folding energy potential of the Herpes Simplex 1 type 17 genome')
ax1.set_ylabel('Deletions', color='g')
ax2.set_ylabel('Folding energy kcal/mol', color='b')